# Case: Criminal location tracking
**Context**

> *During several home searches by the Dutch police, phones were confiscated. After examining these phones, they were found to contain GPS information. It is believed that these two individuals met. Given the importance of the criminal investigation, can you find out the exact spot where these two individuals met?*


source: [https://www.crimediggers.nl/]()

The solutions is broken down into the following steps:
1. Read in the location data
2. Visualize the route of the two criminals on a map
3. Get the exact times and coordinates when they met


<br><details><summary>Hints</summary>

- Sometimes one criminal is standing still for a while, when the other one is moving!
- Animate the route of the criminals during the day, to see where they are at what time.

</div>


In [243]:
# Import libraries
import pandas as pd
import numpy as np

## 1. Read in the location data
- check which columns are in it, and its type
- check if there are any missing values
- check if the data is sorted by time

In [244]:
# Read in the data in the /data/ folder
gps1 = pd.read_csv('./data/GPS_1.csv', parse_dates=['datetime'])
gps2 = pd.read_csv('./data/GPS_2.csv', parse_dates=['datetime'])

## 2. Visualize the route of the two criminals on a map
For data visualisation, the `plotly.express` libary is a great choice. It is a high-level library that is widely used. You can find documentation [here](https://plotly.com/python/).


<br><details><summary>Hint 1</summary>

- use `px.scatter_mapbox()` to create a map
- check the parameters of `px.scatter_mapbox()` function to customize your map
</details> 

</div>

<br><details><summary>Hint 2</summary>

- You can animate the map by adding the parameter `animation_frame` to the `px.scatter_mapbox()` function

</div>

In [245]:
import plotly.express as px
import plotly.graph_objects as go

### Criminal 1

In [246]:
# Make sure the data is sorted by time (if not already sorted)
# Create an animated map using Plotly
gps1['time'] = gps1['datetime'].dt.time
gps2['time'] = gps2['datetime'].dt.time

fig = px.scatter_mapbox(
    gps1,
    lat='latitude',
    lon='longitude',
    hover_name='time',
    mapbox_style='carto-positron',
    title='Route of criminal 1 in Amsterdam',
    zoom=13
)

# Draw lines to connect the points in the order of time
fig.update_traces(line=dict(color='blue', width=2), selector=dict(mode='lines+markers'))

# Customize the appearance of the map if needed
fig.update_layout(
    height=600,
    margin={'r': 0, 't': 40, 'l': 0, 'b': 0},
    autosize=True,
)
fig.show()

### Criminal 2
Copy your code from the previous cell and change the dataframe to `gps2` to plot the route of criminal 2.

In [247]:
fig = px.scatter_mapbox(
    gps2,
    lat='latitude',
    lon='longitude',
    hover_name='time',
    mapbox_style='carto-positron',
    title='Route of criminal 2 in Amsterdam',
    zoom=13,)

# Draw lines to connect the points in the order of time
fig.update_traces(line=dict(color='blue', width=2), selector=dict(mode='lines+markers'))

# Customize the appearance of the map if needed
fig.update_layout(
    height=600,
    margin={'r': 0, 't': 40, 'l': 0, 'b': 0},
    autosize=True,
)
fig.show()

### Plot route of both criminals on one map to see which places they have in common. 

<br><details><summary>Hint</summary>
    - add a new column to each dataframe with the name of the criminal
    - use `pd.concat()` to merge the two dataframes
    - use `px.line_mapbox()` to create a map
    
</div>

<br><details><summary>Hint</summary>
    - use the `color` parameter to color the routes differently
    
</div>

In [248]:
# Add a column with the name of the criminal
gps1['suspect'] = '1'
gps2['suspect'] = '2'

# Take the two dataframes together in one dataframe
gps = pd.concat([gps1, gps2], axis=0)
gps = gps.sort_values(by='datetime')
gps['time'] = gps['datetime'].dt.time

# Visualize both routes on one map using the 'color' argument
fig = px.line_mapbox(
    gps,
    lat='latitude',
    lon='longitude',
    mapbox_style='carto-positron',
    hover_name='datetime',
    title='Route of criminals in Amsterdam',
    color='suspect',
    zoom=13,
)

# change height
fig.update_layout(
    height=600, 
    margin={'r': 0, 't': 40, 'l': 0, 'b': 0},
)


As you can see, the two criminals share common places. But when where they at the same place **at the same time?** You can manually check for each point, but that is not very efficient. Let's try to find a more efficient way to do this.

## 3. Get the exact time and coordinates when they met

We want to use the `merge_ordered` function of pandas to take the gps data together. It merges the two gps location data together on the datetime column. The fill_method parameter of the function ensures that the last known location of suspect 2 is put together with the location of suspect 1. 


In [252]:
# Calculate the distance between the two criminals for each point in time
# To calculate the distance between two gps points, we can use the geodesic fuction of the geopy library
from geopy.distance import geodesic

# First merge_asof gps1 and gps2 on datetime
gps1 = gps1.sort_values(by='datetime')
gps2 = gps2.sort_values(by='datetime')

gps_merged = pd.merge_ordered(gps1, gps2, on='datetime', fill_method='ffill', suffixes=['_suspect_1', '_suspect_2'])
gps_merged[['longitude_suspect_1', 'latitude_suspect_1', 'longitude_suspect_2', 'latitude_suspect_2']] = gps_merged[['longitude_suspect_1', 'latitude_suspect_1', 'longitude_suspect_2', 'latitude_suspect_2']].fillna(0)

# New column with the distance between the two criminals (in meters)
gps_merged['distance'] = gps_merged.apply(lambda x: geodesic((x['latitude_suspect_1'], x['longitude_suspect_1']), (x['latitude_suspect_2'], x['longitude_suspect_2'])).meters, axis=1)

,latitude_suspect_1,longitude_suspect_1,datetime,time_suspect_1,suspect_suspect_1,latitude_suspect_2,longitude_suspect_2,time_suspect_2,suspect_suspect_2,distance
289,52.360531,4.874528,2019-11-05 17:11:00+01:00,16:51:59,1,52.360529,4.874534,17:11:00,2,0.47198


## 4. Solution
At 16:15 and 16:30, Suspect 1 and Suspect 2 moved respectively in direction of Vondelpark. But while Suspect 1 passed directly through Vondelstraat, Suspect 2 preferred a longer way to go via Overtoom street (which is longer). Suspect 1 stopped at 16:50 at the spot in RED in the Figure waiting for Suspect 2. He probably turned off his phone then (hence the time gap in the data). Suspect 2 arrived at the same spot around 17:10. He moved immediately to home which reached he reached at 18:00.

In [264]:
# Get point in time when the two criminals were closest to each other
meeting_point = gps_merged[gps_merged['distance'] == gps_merged['distance'].min()]

print(f'The two criminals were closest to each other at {meeting_point["datetime"].iloc[0].time()}, at {round(meeting_point["distance"].iloc[0], 2)} meters from each other.')

The two criminals were closest to each other at 17:11:00, at 0.47 meters from each other.


In [258]:
# Plot the meeting point on a map
fig = px.scatter_mapbox(
    gps,
    lat='latitude',
    lon='longitude',
    hover_name='time',
    mapbox_style='carto-positron',
    title='Meeting point',
    zoom=17,
    color='suspect',
    center={'lat': 52.3605, 'lon': 4.8745},
    height=800,
)

# Draw a circle around meeting point 
fig.add_trace(
    go.Scattermapbox(
        lat=[52.360531],
        lon=[4.874528],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=60,
            color='orange',
            opacity=0.4
        ),
        text=['Meeting point'],
        hoverinfo='text'
    )
)

fig.show()